In [1]:
from IPython.display import clear_output
from datetime import datetime

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

import time

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import re

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

import html as html_lib

In [2]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：2



,No,Url,Request_Url
0,20,https://www.amazon.com/dp/B00HQ2CCCE,https://www.amazon.com/dp/B00HQ2CCCE?language=...
1,81,https://www.amazon.com/dp/B0C3ML1NDC,https://www.amazon.com/dp/B0C3ML1NDC?language=...


In [3]:
import time

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
# option.add_argument('--headless=new')
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument(get_header(method='selenium'))
option.add_argument(get_proxy(method='selenium'))
option.page_load_strategy = 'normal'

driver = Chrome(service=Service('../chromedriver'), options=option)
driver.maximize_window()

# = = =  = = =  = = =

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        driver.get('https://www.amazon.com/?language=en_US')
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

driver

1


<selenium.webdriver.chrome.webdriver.WebDriver (session="350cae58dfed37274fdad77b8424a038")>

In [4]:
for crawler_index, crawler_series in df_input.iterrows():

    crawler_status = False

    # = = =  = = =  = = =

    try:
        url_parse = urlparse(crawler_series['Url'])

        dict_param = parse_qs(url_parse.query)
        dict_param['language'] = 'en_US'

        url_request = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

        crawler_retry = 0
        while True:
            crawler_retry += 1

            try:
                driver.get(url_request)
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
                time.sleep(1.5)
                driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
                time.sleep(0.7)
                driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
                time.sleep(1.2)
                driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, 0);")

                soup = BeautifulSoup(driver.page_source, 'lxml')
                html = etree.HTML(str(soup))

                break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = =  = = =  = = =

        zip_code = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', html.xpath('//span[@id="glow-ingress-line2"]/text()')[0]).strip()

        # = = =  = = =  = = =

        price = html.xpath('//span[@class="a-offscreen"]/text()')[0].strip() if html.xpath('//span[@class="a-offscreen"]/text()') else ''

        # = = =  = = =  = = =

        dict_src, list_src = {}, json.loads('{"initial":[{' + str(soup).split('\'colorImages\': { \'initial\': [{')[1].split('}]},')[0] + '}]}')['initial']
        for i, src in enumerate(list_src):
            dict_src[str(i)] = src['hiRes'].strip() if src['hiRes'] else src['large'].strip()

        # = = =  = = =  = = =

        dict_key, list_tr = {}, html.xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr')
        for tr in list_tr:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
            if key in dict_key:
                dict_key[key] += 1
            else:
                dict_key[key] = 1

        dict_overview, dict_count = {}, {}
        for i, tr in enumerate(list_tr):
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
            if dict_key[key] == 1:
                dict_overview[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}
            else:
                if key in dict_key:
                    dict_key[key] += 1
                else:
                    dict_key[key] = 1
                dict_overview[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}

        # = = =  = = =  = = =

        about, list_text = '', [re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', text).strip() for text in html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')]
        for i, text in enumerate(list_text):
            about += f'{i + 1}. {text}\n'

        # = = =  = = =  = = =

        dict_ = json.loads(re.sub(r'\\(?!["\\])', r'\\\\', str(soup).split('jQuery.parseJSON(\'')[1].split('}\');')[0] + '}'))

        # = = =  = = =  = = =

        title = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', dict_['title']).strip()

        # = = =  = = =  = = =

        parent_asin = dict_['parentAsin'].strip()

        # = = =  = = =  = = =

        dict_key, list_tr = {}, html.xpath('//table[@id="productDetails_techSpec_section_1"]/tbody/tr')
        for tr in list_tr:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
            if key in dict_key:
                dict_key[key] += 1
            else:
                dict_key[key] = 1

        dict_technical, dict_count = {}, {}
        for i, tr in enumerate(list_tr):
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
            if dict_key[key] == 1:
                dict_technical[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
            else:
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
                dict_technical[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

        # = = =  = = =  = = =

        i, dict_additional, list_tr = 0, {}, html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tbody/tr')
        for tr in list_tr:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
            if key == 'Customer Reviews':
                dict_additional[str(i)] = {'Customer Reviews Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
                dict_additional[str(i + 1)] = {'Customer Reviews Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
                i += 2
            elif key == 'Best Sellers Rank':
                list_span = tr.xpath('./td/span/span')
                list_text = [etree.tostring(span).decode('utf-8') for span in list_span]
                list_text = [re.sub(r'<.*?>|\(.*?\)', '', text).strip() for text in list_text]
                list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
                list_text = [html_lib.unescape(text) for text in list_text]

                list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
                for j, (text, href) in enumerate(zip(list_text, list_href)):
                    if j == 0:
                        dict_additional[str(i)] = {'Best Sellers Rank': text}
                        dict_additional[str(i + 1)] = {'Best Sellers Rank Url': href}
                    else:
                        dict_additional[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                        dict_additional[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
                    i += 2
            else:
                dict_additional[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
                i += 1

        # = = =  = = =  = = =

        dict_key, list_tr = {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/div[@id="detailBullets_feature_div"]/ul/li/span')
        for tr in list_tr:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
            if key in dict_key:
                dict_key[key] += 1
            else:
                dict_key[key] = 1

        dict_technical_2, dict_count = {}, {}
        for i, tr in enumerate(list_tr):
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
            if dict_key[key] == 1:
                dict_technical_2[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}
            else:
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
                dict_technical_2[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}

        # = = =  = = =  = = =

        i, dict_additional_2, list_span = 0, {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]/li/span')
        for span in list_span:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
            if key == 'Customer Reviews':
                dict_additional_2[str(i)] = {'Customer Reviews Rating Count': span.xpath('./div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
                dict_additional_2[str(i + 1)] = {'Customer Reviews Rating': span.xpath('./div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
                i += 2
            elif key == 'Best Sellers Rank':
                list_span_2 = span.xpath('./descendant-or-self::span[@class="a-list-item"]')
                list_text = [etree.tostring(span_2).decode('utf-8') for span_2 in list_span_2]
                list_text = [re.sub(r'<ul.*?</ul>|<span.*?Best Sellers Rank.*?</span>|<.*?>|\(.*?\)', '', text).strip() for text in list_text]
                list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
                list_text = [html_lib.unescape(text) for text in list_text]

                list_href = [f'https://www.amazon.com{href.strip()}' for href in span.xpath('./descendant-or-self::a/@href')]
                for j, (text, href) in enumerate(zip(list_text, list_href)):
                    if j == 0:
                        dict_additional_2[str(i)] = {'Best Sellers Rank': text}
                        dict_additional_2[str(i + 1)] = {'Best Sellers Rank Url': href}
                    else:
                        dict_additional_2[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                        dict_additional_2[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
                    i += 2
            else:
                print('fuck')

        # = = =  = = =  = = =

        dict_key, list_tr = {}, html.xpath('//div[@id="productDetails_expanderSectionTables"]/div[@class="a-row"]/div[contains(@class, "a-column a-span6")]/div[@class="a-row a-spacing-medium-plus"]/div[@class="a-section"]/div[contains(@data-csa-c-content-id, "voyager-expander-btn")]/div/div[@class="a-section"]/table/tbody/tr')
        for tr in list_tr:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
            if key in dict_key:
                dict_key[key] += 1
            else:
                dict_key[key] = 1

        i, dict_information, dict_count = 0, {}, {}
        for tr in list_tr:
            key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
            if dict_key[key] == 1:
                if key == 'Customer Reviews':
                    dict_information[str(i)] = {'Customer Reviews Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
                    dict_information[str(i + 1)] = {'Customer Reviews Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
                    i += 2
                elif key == 'Best Sellers Rank':
                    list_span = tr.xpath('./td/span/span')
                    list_text = [etree.tostring(span).decode('utf-8') for span in list_span]
                    list_text = [re.sub(r'<.*?>|\(.*?\)', '', text).strip() for text in list_text]
                    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
                    list_text = [html_lib.unescape(text) for text in list_text]

                    list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
                    for j, (text, href) in enumerate(zip(list_text, list_href)):
                        if j == 0:
                            dict_information[str(i)] = {'Best Sellers Rank': text}
                            dict_information[str(i + 1)] = {'Best Sellers Rank Url': href}
                        else:
                            dict_information[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                            dict_information[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
                        i += 2
                else:
                    dict_information[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
                    i += 1
            else:
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
                dict_information[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

        # = = =  = = =  = = =

        if not dict_technical and not dict_additional and not dict_technical_2 and not dict_additional_2 and not dict_information:
            raise

        # = = =  = = =  = = =

        list_dict_correct.append({'No': crawler_series['No'],
                                  'Title': title,
                                  'About': about.strip(),
                                  'Price': price,
                                  'Picture': '',
                                  'Url': url_request,
                                  'Zip Code': zip_code,
                                  'Parent ASIN': parent_asin,
                                  'Json_Src': json.dumps(dict_src),
                                  'Json_Overview': json.dumps(dict_overview),
                                  'Json_Technical': json.dumps(dict_technical),
                                  'Json_Additional': json.dumps(dict_additional),
                                  'Json_Technical 2': json.dumps(dict_technical_2),
                                  'Json_Additional 2': json.dumps(dict_additional_2),
                                  'Json_Information': json.dumps(dict_information)})

        # = = =  = = =  = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        list_series_error.append(crawler_series)

    # = = =  = = =  = = =

    if crawler_remain % 30 == 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

    crawler_remain -= 1

# = = =  = = =  = = =

print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/2.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
print('Output Finished !')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/page_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

# = = =  = = =  = = =

driver.quit()
print('Done ~')

[True] - [请求1次] - [剩余1条] - [2024/11/30 15:52:47] - 20. https://www.amazon.com/dp/B00HQ2CCCE?language=en_US

[True] - [请求1次] - [剩余0条] - [2024/11/30 15:52:55] - 81. https://www.amazon.com/dp/B0C3ML1NDC?language=en_US

Data Outputting...
Output Finished !

Done ~
